In [98]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd

driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
from selenium.common.exceptions import ElementClickInterceptedException

url = 'https://cellphones.com.vn/laptop.html'
driver.get(url)

#TẠO HÀM KIỂM TRA NÚT XEM THÊM
def is_view_more_button_displayed(selector='.cps-block-content_btn-showmore'):
    try:
        view_more_button = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, selector))
        )
        return view_more_button.is_displayed()
    except:
        return False
#TẠO HÀM TỰ ĐỘNG CUỘN TRANG
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView({ behavior: 'smooth', block: 'center', inline: 'nearest' });", element)

#TẠO HÀM SO SÁNH SỐ LƯỢNG SẢN PHẨM TRÊN TRANG
def is_product_count_changed(previous_product_count):
    current_product_count = len(driver.find_elements(By.CLASS_NAME, 'product-info'))
    return current_product_count != previous_product_count

# Lưu số lượng sản phẩm ban đầu
initial_product_count = len(driver.find_elements(By.CLASS_NAME, 'product-info'))

# Vòng lặp để click vào nút "Xem thêm" cho đến khi nó không còn hiển thị
while is_view_more_button_displayed('.cps-block-content_btn-showmore'):
    initial_product_count = len(driver.find_elements(By.CLASS_NAME, 'product-info'))
    view_more_button = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.cps-block-content_btn-showmore'))
    )
    # Scroll đến phần tử mục tiêu trước khi kiểm tra hiển thị
    scroll_to_element(view_more_button)

    try:
        view_more_button.click()
    except ElementClickInterceptedException:
        # Xử lý pop-up
        close_popup_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.cancel-button-top'))
        )
        close_popup_button.click()
        # Thực hiện thao tác click lại
        view_more_button.click()
    sleep(3)

    # Kiểm tra xem số lượng sản phẩm có thay đổi hay không
    if not is_product_count_changed(initial_product_count):
        break
html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')
product_items = soup.find_all("div", class_="product-info-container")

# Khởi tạo list để lưu trữ thông tin sản phẩm
product_list = []

# Lặp qua từng sản phẩm và lấy thông tin
for product_item in product_items:
    product_name = product_item.find("div", class_="product__name").text.strip()
    discounted_price = product_item.find("p", class_="product__price--show").text.strip()
    try:
        original_price = product_item.find("p", class_="product__price--through").text.strip()
    except:
        original_price = discounted_price
    try:
        product_discount = product_item.find("p", class_="product__price--percent-detail").text.strip()
    except:
        product_discount = ''
    # Lấy thông tin khuyến mãi
    promotions = product_item.find_all("div", class_="product__promotions")
    product_promotions = ', '.join(promotion.text.strip() for promotion in promotions if promotion.text.strip())
    try:
        product_url = product_item.find('a')['href']
    except:
        product_url = ''
    try:
        installment_payment = product_item.find('div', class_="install-0-tag").text.strip()
    except: 
        installment_payment = ''
    
    # Tạo dictionary chứa thông tin sản phẩm
    product_info = {
        'product_name': product_name,
        'discounted_price': discounted_price,
        'original_price': original_price,
        'discount_percentage': product_discount,
        'installment_payment' : installment_payment,
        'promotions': product_promotions,
        'shop_brand' : 'cellphoneS',
        'product_url': product_url
    }
    # Thêm dictionary vào list
    product_list.append(product_info)

driver.quit()

In [101]:
import pandas as pd
df = pd.DataFrame(product_list)
df.head(10)

,product_name,original_price,discounted_price,discount_percentage,installment_payment,promotions,shop_brand,product_url
0,Laptop Asus TUF Gaming F15 FX506HF-HN078W,20.990.000₫,17.190.000₫,Giảm 18%,Trả góp 0%,,cellphoneS,/laptop-asus-tuf-gaming-f15-fx506hf-hn078w.html
1,Laptop Asus VivoBook S 14 FLIP TP3402VA-LZ025W,16.990.000₫,11.990.000₫,Giảm 29%,Trả góp 0%,,cellphoneS,/laptop-asus-vivobook-s-14-flip-tp3402va-lz025...
2,Laptop MSI Gaming Bravo 15 B7ED-010VN,18.490.000₫,16.490.000₫,Giảm 11%,Trả góp 0%,,cellphoneS,/laptop-msi-gaming-bravo-15-b7ed-010vn.html
3,Laptop HP Pavilion 14-DV2073TU 7C0P2PA,20.990.000₫,16.490.000₫,Giảm 21%,Trả góp 0%,,cellphoneS,/laptop-hp-pavilion-14-dv2073tu-7c0p2pa.html
4,Laptop HP Gaming Victus 15-FB1023AX 94F20PA,24.490.000₫,17.590.000₫,Giảm 28%,Trả góp 0%,,cellphoneS,/laptop-hp-gaming-victus-15-fb1023ax-94f20pa.html
5,Laptop Lenovo IdeaPad 3 14ITL6 82H701QWVN,11.290.000₫,9.390.000₫,Giảm 17%,Trả góp 0%,,cellphoneS,/laptop-lenovo-ideapad-3-14itl6-82h701qwvn.html
6,Laptop Acer Aspire 3 A315-58-53S6 NX.AM0SV.005,15.990.000₫,10.790.000₫,Giảm 33%,Trả góp 0%,,cellphoneS,/laptop-acer-aspire-3-a315-58-53s6.html
7,Laptop Huawei Matebook D15,13.990.000₫,11.490.000₫,Giảm 18%,Trả góp 0%,,cellphoneS,/laptop-huawei-matebook-d15-r7-5700u.html
8,Laptop MSI Modern 15 B12MO-628VN,14.490.000₫,12.790.000₫,Giảm 12%,Trả góp 0%,,cellphoneS,/laptop-msi-modern-15-b12mo-628vn.html
9,Laptop Lenovo Ideapad Slim 5 14IAH8 83BF002NVN,17.290.000₫,14.790.000₫,Giảm 14%,Trả góp 0%,,cellphoneS,/laptop-lenovo-ideapad-slim-5-14iah8-83bf002nv...


In [100]:
df.to_excel('cellphones_price.xlsx', index=False)